In [ ]:
# Initialisation
import twitter
import pandas as pd
import re
import json
import numpy as np

with open('../credential/bearer_token.txt') as f:
    BEARER_TOKEN = f.read()
t_h = twitter.Twitter(
    auth=twitter.OAuth2(bearer_token=BEARER_TOKEN)
    )

In [ ]:
# For loop preparation
squid_df = None
max_iter = 30
since_id_buff = ''

# Loop. Expect 25 seconds run for 10 iterations, yielding 1000 tweets.
for i in range(max_iter):
    query = t_h.search.tweets(q='Squid game', lang='en', count=100, tweet_mode='extended') if since_id_buff == '' else t_h.search.tweets(q='Squid game', lang='en', count=100, tweet_mode='extended', max_id=since_id_buff)
    since_id_buff = re.search(r'max_id=(.+)&q', query['search_metadata']['next_results']).group(1)
    query = pd.DataFrame(query['statuses'])
    squid_df = squid_df.append(query).reset_index().drop('index', axis=1) if squid_df is not None else query
    

In [ ]:
squid_df.tail() # by now, we should have 3000 tweets.

In [ ]:
# Clean duplicates
squid_df = squid_df.drop_duplicates(subset=['full_text']).reset_index().drop('index', axis=1)
squid_df.tail()

In [ ]:
# JSON flatten
squid_df_retweet = []

# pd.isna(squid_df['retweeted_status'].iloc[2])
for i in squid_df['retweeted_status']:
    try:
        squid_df_retweet.append(i['full_text'])
    except:
        squid_df_retweet.append(np.nan)

# squid_df_retweet = 

In [ ]:
pd.isnull(squid_df_retweet[4])

In [ ]:
# squid_df_retweet # shows the longer tweets (in cases of retweets)
squid_tweets = []

for i in range(len(squid_df_retweet)):
    squid_tweets.append(squid_df_retweet[i] if not pd.isnull(squid_df_retweet[i]) else squid_df['full_text'].loc[i])

pass
squid_tweets

In [ ]:
# Saving to txt in dataset
output_dataset = '../dataset/squid-tweets/squid_tweets.json'

# for i in squid_tweets:
    # print(i)

with open(output_dataset, 'w') as f:
    f.write(json.dumps(squid_tweets))

